# Example of KEO Taylor series expansion for ammonia

Here, we use the following internal coordinates

1. $r_1$
2. $r_2$
3. $r_3$
4. $s_4 = (2\alpha_{23}-\alpha_{13}-\alpha_{12})/\sqrt{6}$
5. $s_5 = (\alpha_{13}-\alpha_{12})/\sqrt{2}$
6. $\rho$

where $r_i\equiv\text{N-H}_i$, $\alpha_{ij}\equiv\text{H}_i\text{-C-H}_j$, and $\rho$ is an 'umbrella' angle $[0,\pi]$.

We will build expansion of the potential energy surface (PES) in terms of the following 1D functions of internal coordinates:

- $y_1=1-\exp(-a_m(r_1-r_1^{(eq)}))$
- $y_2=1-\exp(-a_m(r_2-r_2^{(eq)}))$
- $y_3=1-\exp(-a_m(r_3-r_3^{(eq)}))$
- $y_4=s_4$
- $y_5=s_5$
- $y_6=\cos(\rho)$

For expansion of the kinetic energy operator (KEO), we use the following 1D functions of internal coordinates:

- $z_1=r_1-r_1^{(eq)}$
- $z_2=r_2-r_2^{(eq)}$
- $z_3=r_3-r_3^{(eq)}$
- $z_4=s_4$
- $z_5=s_5$
- $z_6=\cos(\rho)$

In [1]:
import itertools
import os

import jax
import matplotlib.pyplot as plt
import numpy as np
from jax import config
from jax import numpy as jnp

from vibrojet.keo import Gmat, com, pseudo
from vibrojet.potentials import nh3_POK
from vibrojet.taylor import deriv_list
from vibrojet.jet_prim import acos

config.update("jax_enable_x64", True)

Define a function `find_alpha_from_s_delta` to obtain three $\alpha_{ij}$ valence angular coordinates from the two symmetrized $s_4$, $s_5$ and 'umbrella' angle $\delta=\rho-\pi/2$ coordinates.

In [2]:
def find_alpha_from_s_delta(s4, s5, delta, no_iter: int = 20):

    def calc_s_to_sin_delta(s6, s4, s5):
        alpha1 = (jnp.sqrt(2) * s6 + 2.0 * s4) / jnp.sqrt(6)
        alpha2 = (jnp.sqrt(2) * s6 - s4 + jnp.sqrt(3) * s5) / jnp.sqrt(6)
        alpha3 = (jnp.sqrt(2) * s6 - s4 - jnp.sqrt(3) * s5) / jnp.sqrt(6)
        tau_2 = (
            1
            - jnp.cos(alpha1) ** 2
            - jnp.cos(alpha2) ** 2
            - jnp.cos(alpha3) ** 2
            + 2 * jnp.cos(alpha1) * jnp.cos(alpha2) * jnp.cos(alpha3)
        )
        norm_2 = (
            jnp.sin(alpha3) ** 2
            + jnp.sin(alpha2) ** 2
            + jnp.sin(alpha1) ** 2
            + 2 * jnp.cos(alpha3) * jnp.cos(alpha1)
            - 2 * jnp.cos(alpha2)
            + 2 * jnp.cos(alpha2) * jnp.cos(alpha3)
            - 2 * jnp.cos(alpha1)
            + 2 * jnp.cos(alpha2) * jnp.cos(alpha1)
            - 2 * jnp.cos(alpha3)
        )
        return tau_2 / norm_2

    # initial value for s6
    alpha1 = 2 * jnp.pi / 3
    s6 = alpha1 * jnp.sqrt(3)
    sin_delta = jnp.sin(delta)

    for _ in range(no_iter):
        f = calc_s_to_sin_delta(s6, s4, s5)
        eps = f - sin_delta**2
        grad = jax.grad(calc_s_to_sin_delta)(s6, s4, s5)
        dx = eps / grad
        dx0 = dx
        s6 = s6 - dx0

    alpha1 = (jnp.sqrt(2) * s6 + 2 * s4) / jnp.sqrt(6)
    alpha2 = (jnp.sqrt(2) * s6 - s4 + jnp.sqrt(3) * s5) / jnp.sqrt(6)
    alpha3 = (jnp.sqrt(2) * s6 - s4 - jnp.sqrt(3) * s5) / jnp.sqrt(6)

    return alpha1, alpha2, alpha3

Define mapping from internal coordinates to Cartesian coordinates, to $y$-coordinates used for expansion of PES, and $z$-coordinates used for expansion of KEO.

In [3]:
# masses of N, H1, H2, H3
masses = [14.00307400, 1.007825035, 1.007825035, 1.007825035]

# reference values of internal coordinates
r_ref = 1.01031310  # in Angstrom
rho_ref = np.pi / 2
q0 = [r_ref, r_ref, r_ref, 0.0, 0.0, rho_ref]

# Morse constant necessary for defining y-coordinates for stretches
a_morse = 2.15


@com(masses)
def internal_to_cartesian(internal_coords):
    r1, r2, r3, s4, s5, rho = internal_coords
    delta = rho - jnp.pi / 2
    alpha1, alpha2, alpha3 = find_alpha_from_s_delta(s4, s5, delta)

    beta3 = jnp.acos((jnp.cos(alpha3) - jnp.cos(rho) ** 2) / jnp.sin(rho) ** 2)
    beta2 = jnp.acos((jnp.cos(alpha2) - jnp.cos(rho) ** 2) / jnp.sin(rho) ** 2)
    cartesian = jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.sin(rho), 0.0, r1 * jnp.cos(rho)],
            [
                r2 * jnp.sin(rho) * jnp.cos(beta3),
                r2 * jnp.sin(rho) * jnp.sin(beta3),
                r2 * jnp.cos(rho),
            ],
            [
                r3 * jnp.sin(rho) * jnp.cos(beta2),
                -r3 * jnp.sin(rho) * jnp.sin(beta2),
                r3 * jnp.cos(rho),
            ],
        ]
    )
    return cartesian


def internal_to_y(q):
    r1, r2, r3, s4, s5, rho = q
    y1 = 1 - jnp.exp(-a_morse * (r1 - r_ref))
    y2 = 1 - jnp.exp(-a_morse * (r2 - r_ref))
    y3 = 1 - jnp.exp(-a_morse * (r3 - r_ref))
    y4 = s4
    y5 = s5
    y6 = jnp.cos(rho)
    return jnp.array([y1, y2, y3, y4, y5, y6])


def y_to_internal(y):
    y1, y2, y3, y4, y5, y6 = y
    r1 = -jnp.log(1 - y1) / a_morse + r_ref
    r2 = -jnp.log(1 - y2) / a_morse + r_ref
    r3 = -jnp.log(1 - y3) / a_morse + r_ref
    s4 = y4
    s5 = y5
    rho = acos(y6)
    return jnp.array([r1, r2, r3, s4, s5, rho])


y0 = internal_to_y(q0)

In [4]:
xyz = internal_to_cartesian(q0)
print("Reference values of internal coordinates:\n", q0)
print("Reference values of expansion y-coordinates:\n", y0)
print("Reference values of Cartesian coordinates:\n", xyz)

Reference values of internal coordinates:
 [1.0103131, 1.0103131, 1.0103131, 0.0, 0.0, 1.5707963267948966]
Reference values of expansion y-coordinates:
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 6.123234e-17]
Reference values of Cartesian coordinates:
 [[ 2.85580222e-17  2.96388443e-19 -1.09854184e-17]
 [ 1.01031310e+00  2.96388443e-19  5.08784168e-17]
 [-5.05156550e-01  8.74956810e-01  5.08784168e-17]
 [-5.05156550e-01 -8.74956810e-01  5.08784168e-17]]


Generate expansion power indices

In [5]:
max_order = 6  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(q0))])
    if sum(elem) <= max_order
]
print("max expansion order:", max_order)
print("number of expansion terms:", len(deriv_ind))

max expansion order: 6
number of expansion terms: 924


Generate expansion of PES in terms of $y$-coordinates

In [8]:
@jax.jit
def poten_in_y(y):
    q = y_to_internal(y)
    r1, r2, r3, s4, s5, rho = q
    delta = rho - jnp.pi / 2
    alpha1, alpha2, alpha3 = find_alpha_from_s_delta(s4, s5, delta)
    v = nh3_POK.poten((r1, r2, r3, alpha1, alpha2, alpha3))
    return v

poten_file = f"_nh3_poten_coefs_{max_order}.npy"
if os.path.exists(poten_file):
    print(f"load potential expansion coefs from file {poten_file}")
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(poten_in_y, deriv_ind, y0, if_taylor=True)
    np.save(poten_file, poten_coefs)

load potential expansion coefs from file _nh3_poten_coefs_6.npy


Generate expansion of KEO in terms of $z$-coordinates

Test if the original potential `nh3_POK.poten` at planar equilibrium configuration has the same value as the zero-order expansion coefficient in `poten_coefs`

In [9]:
alpha_ref = 120 * np.pi / 180
pot_planar = nh3_POK.poten((r_ref, r_ref, r_ref, alpha_ref, alpha_ref, alpha_ref))

coef0 = poten_coefs[deriv_ind.index(tuple([0] * 6))]

print(pot_planar, coef0, pot_planar - coef0)

1930.469934890773 1930.4699348907843 -1.1368683772161603e-11
